### Air Pollution Study Using EPA Data

The study below looks at air quality in the U.S. between 2004 and 2014 using selected data from the EPA, available at http://aqsdr1.epa.gov/aqsweb/aqstmp/airdata/download_files.html. Specifically, we look at PM 2.5 data, which refers to particulate matter less than 2.5 microns in size.

The goal is to see how pollution levels have changed during the period of interest, both nationwide and for individual states.


In [ ]:
import numpy as np
import pandas as pd
